In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data

logging.getLogger('frankenfit').setLevel(logging.INFO)

In [2]:
import frankenfit as ff

In [3]:
import frankenfit.core as core
reload(core)
import frankenfit.universal_transforms as ut
reload(ut)

<module 'frankenfit.universal_transforms' from '/home/max/frankenfit/src/frankenfit/universal_transforms.py'>

In [4]:
class AddFoo(core.StatelessTransform):
    def _apply(self, df_apply: pd.DataFrame, state: object = None) -> pd.DataFrame:
        return df_apply.assign(foo=1)

AddFoo().apply()

,foo


In [5]:

core.BasePipeline.with_methods()

frankenfit.core.BasePipeline.with_methods.<locals>.BasePipelineWithMethods

In [4]:
core.BasePipeline(tag="foo")

BasePipeline(tag='foo', transforms=[])

In [6]:
core.BasePipeline.with_methods(identity=ut.Identity)(tag="foo").identity()

BasePipelineWithMethods(tag='foo', transforms=[Identity(tag='Identity#1')])

In [30]:
BPWM = core.BasePipeline.with_methods(identity=ut.Identity)
BPWM(tag="foo").identity().identity()

BasePipelineWithMethods(tag='foo', transforms=[Identity(tag='Identity#2'), Identity(tag='Identity#3')])

In [32]:
class MyPipeline(BPWM):
    pass

MyPipeline().identity().identity()

MyPipeline(tag='MyPipeline#1', transforms=[Identity(tag='Identity#6'), Identity(tag='Identity#7')])

In [33]:
ut.Identity() + MyPipeline().identity().identity()

Pipeline(tag='Pipeline#1', transforms=[Identity(tag='Identity#8'), Identity(tag='Identity#9'), Identity(tag='Identity#10')])

In [5]:
class Identity(core.StatelessTransform):
    def _apply(self, df_apply: pd.DataFrame, state: object = None):
        return df_apply

class MyPipeline(core.BasePipeline.with_methods(
    identity=Identity
)):
    pass

Identity.pipeline_type = MyPipeline

#MyPipeline().identity().identity()
Identity() + Identity()

MyPipeline(tag='MyPipeline#1', transforms=[Identity(tag='Identity#1'), Identity(tag='Identity#2')])

In [37]:
(Identity() + Identity()).identity()

MyPipeline(tag='MyPipeline#5', transforms=[Identity(tag='Identity#13'), Identity(tag='Identity#14'), Identity(tag='Identity#15')])

In [6]:
x = ut.IfHyperparamIsTrue(
    "do_add_foo",
    AddFoo(),
    otherwise=ut.Print("hello")
)
display(x.fit(bindings={"do_add_foo": True}).apply(pd.DataFrame({"bar":np.arange(10)})))
display(x.fit(bindings={"do_add_foo": False}).apply(pd.DataFrame({"bar":np.arange(10)})))

,bar,foo
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


hello


,bar
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [10]:
df = data('diamonds').reset_index().set_index('index')
pip = core.BasePipeline([
    ut.Identity(),
    ut.Identity()
])

pip.apply(df)

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56


In [11]:
pip

BasePipeline(tag='BasePipeline#2', transforms=[Identity(tag='Identity#9'), Identity(tag='Identity#10')])

In [7]:
ut.Identity().then(ut.Identity())

Pipeline(tag='Pipeline#1', transforms=[Identity(tag='Identity#2'), Identity(tag='Identity#3')])

In [8]:
ut.Identity().then([ut.Identity(), ut.Identity()])

Pipeline(tag='Pipeline#2', transforms=[Identity(tag='Identity#4'), Identity(tag='Identity#5'), Identity(tag='Identity#6')])

In [9]:
ut.Identity() + ut.Identity() + ut.Identity()

Pipeline(tag='Pipeline#3', transforms=[Identity(tag='Identity#7'), Identity(tag='Identity#8'), Identity(tag='Identity#9')])

In [10]:
ut.Pipeline().identity().identity().if_hyperparam_is_true("foo", ut.Pipeline().identity())

Pipeline(tag='Pipeline#4', transforms=[Identity(tag='Identity#10'), Identity(tag='Identity#11'), IfHyperparamIsTrue(tag='IfHyperparamIsTrue#1', name='foo', then=Pipeline(tag='Pipeline#5', transforms=[Identity(tag='Identity#12')]), otherwise=None, allow_unresolved=False)])

In [9]:
p = ut.Pipeline()

In [ ]:
p.id